In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d gspmoreira/articles-sharing-reading-from-cit-deskdrop

In [ ]:
!ls

In [ ]:
!unzip articles-sharing-reading-from-cit-deskdrop.zip

In [ ]:
!ls

In [ ]:
import pandas as pd
import numpy as np

df1 = pd.read_csv("shared_articles.csv")
df2 = pd.read_csv("users_interactions.csv")


In [ ]:
df1.head()

In [ ]:
df2.head()

In [ ]:
#removing content removed
df1 = df1[df1['eventType'] == 'CONTENT SHARED']
df1.head()
df1.shape
df2.shape

In [ ]:
def find_total_events(df1_row):
  total_likes = df2[(df2["contentId"] == df1_row["contentId"]) & (df2["eventType"] == "LIKE")].shape[0]
  total_views = df2[(df2["contentId"] == df1_row["contentId"]) & (df2["eventType"] == "VIEW")].shape[0]
  total_bookmarks = df2[(df2["contentId"] == df1_row["contentId"]) & (df2["eventType"] == "BOOKMARK")].shape[0]
  total_follows = df2[(df2["contentId"] == df1_row["contentId"]) & (df2["eventType"] == "FOLLOW")].shape[0]
  total_comments = df2[(df2["contentId"] == df1_row["contentId"]) & (df2["eventType"] == "COMMENT CREATED")].shape[0]
  return total_likes + total_views + total_bookmarks + total_follows + total_comments

df1["total_events"] = df1.apply(find_total_events, axis=1)

df1.head()

In [ ]:
df1 = df1.sort_values(['total_events'], ascending=[False])

df1.head()

In [ ]:
def convert_lowerCase(x):
  if isinstance(x, list):
        return x.lower()
  else:
    return ''

df1["title"] = df1["title"].apply(convert_lowerCase)
df1.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df1['title'])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [ ]:
df1 = df1.reset_index()
indices = pd.Series(df1.index, index=df1['contentId'])

In [ ]:
def get_recommendations(contentId, cosine_sim):
    idx = indices[contentId]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return df1['contentId'].iloc[movie_indices]

In [ ]:
get_recommendations(-4110354420726924665, cosine_sim2)

In [ ]:
get_recommendations(-825802007843492131	, cosine_sim2)

In [ ]:
get_recommendations(-3460883803089049921, cosine_sim2)

In [ ]:
get_recommendations(-5850390492300666299, cosine_sim2)

In [ ]:
get_recommendations(-7165716941095296287, cosine_sim2)

In [ ]:
from google.colab import files

df1.to_csv('articles.csv')
files.download('articles.csv')